## Phantom Cholesterol Experiments

```python
expdata = np.array([np.mean(np.array(loadmat(f'/content/20230305_Cholesterol_3/Phantom 1/{wave}nm.mat')['ImgData'])[1][0], axis = 3)[:,:,0] for wave in wave_list])
wave, h, w = expdata.shape
```

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.optimize import nnls
from sklearn.decomposition import FastICA
from matplotlib.colors import ListedColormap
from sklearn.decomposition import NMF
%matplotlib inline
from utils import *
from vca import *

In [ ]:
phantom_list = sorted([int(wave[:3]) for wave in os.listdir('../expdata/20230217 Cholesterol and Blood Mix/') if (wave[-3:] == 'mat' and wave[0] != 'S')])
abs_coeff, legend = {}, ['Blood Mixed', 'Cholesterol', 'Background']
wave_abs = np.load('./data/hbo2hbchpr_57.npy')

f = loadmat("./data/unmix.mat")
X, Y = f['x'], f['y']

for idx, wave in enumerate(np.arange(700, 981, 5)):
    abs_coeff[wave] = (idx, wave_abs[idx])
coeffs = np.vstack([abs_coeff[wave][1] for wave in phantom_list])

phantom_coeffs = np.vstack([abs_coeff[wave][1] for wave in phantom_list])
for i in range(phantom_coeffs.shape[0]):
    phantom_coeffs[i][1] = (phantom_coeffs[i][0] * 0.1) + (phantom_coeffs[i][1] * 0.9)
bloodmixch = phantom_coeffs[:,1:3]
weights_plot(array = bloodmixch, wave_list = phantom_list, legend = ['Blood Mix', 'Cholesterol'], figsize = (8, 6), xticks = phantom_list, title = '')

In [ ]:
phantom_data = np.array([np.array(loadmat(f"./data/blood_ch_phantom/PA_Image_{wave}.mat")['Image_PA']) for wave in phantom_list])
plt.figure(figsize = (24, 16))
for i in range(18):
    plt.subplot(3, 6, i + 1)
    plt.imshow(phantom_data[i], cmap = 'jet')
    plt.clim([0, np.max(phantom_data)])
    plt.title(label = f'{phantom_list[i]}nm')
    plt.colorbar()
plt.show()

In [ ]:
phantom_linear = run_linear_unmixing(phantom_data, bloodmixch)
plot_3d_multiple(Y*1000, X*1000, phantom_linear, title = ['Blood Mix', 'Cholesterol', 'Background'], cmap = 'jet', clim = [0, np.max(phantom_linear)], order = [0, 1, 2])

In [ ]:
for i in range(20):
    print(f"Random State: {i}")
    maps, wts, _ = run_ica(phantom_data, phantom_list, 2, i)
    plot_comps_2d(maps, wave_list, wts, figsize = (10, 3), order = [0, 1])

In [ ]:
maps, wts, model = run_ica(phantom_data, phantom_list, 2, 0)
plot_3d_multiple(Y*1000, X*1000, maps, title = ['Blood Mix', 'Cholesterol'], cmap = 'jet', clim = [0, np.max(maps)], order = [1, 0])

In [ ]:
weights_plot(array = np.linalg.pinv(model.components_)[:,[1, 0]], wave_list = phantom_list, legend = ['Blood Mix', 'Cholesterol'], figsize = (8, 6), xticks = phantom_list, title = '')

In [ ]:
os.listdir('./expdata/')

In [ ]:
path = './expdata/20230329 Cholesterol and Blood Mix/'
wavefiles = [wavefile for wavefile in os.listdir(path) if wavefile[-3:] == 'mat' and len(wavefile) == 9]
wavefiles = sorted(wavefiles, key = lambda filename: int(filename[:3]))
pa_cmap = ListedColormap(loadmat(f'{path}{wavefiles[0]}')['PA_colormap'])

In [ ]:
expdata = np.array([np.mean(np.array(loadmat(f'./expdata/20230315 Cholesterol and Blood Mix/{wave}nm.mat')['ImgData'])[1][0], axis = 3)[:,:,0] for wave in phantom_list])
wave, h, w = expdata.shape

In [ ]:
plt.figure(figsize = (30, 16))
for i in range(wave):
    plt.subplot(3, 6, i+1)
    plt.imshow(expdata[i], cmap = "hot")
    plt.title(label = f'{wave_list[i]}nm')
    plt.colorbar()
plt.show()

In [ ]:
linear_comps = run_linear_unmixing(expdata, bloodmixch)
plot_comps_2d(linear_comps, wave_list, bloodmixch, "Linear Unmixing", (20, 4), order = [0, 1], chrom = ['Blood Mix', "Cholesterol"], clim = [0, np.max(linear_comps)])

In [ ]:
test_data = expdata.reshape((wave, h*w))
test_data_mean = test_data.mean(axis = -1)
test_data -= test_data_mean[:, np.newaxis]
test_comps = np.linalg.multi_dot([model._unmixing, model.whitening_, test_data]).T.reshape((h, w, 2))
plot_comps_2d(test_comps, wave_list, np.linalg.pinv(model.components_), "ICA", (20, 4), order = [1, 0], invert_sign = None, clim = [0, np.max(test_comps)], chrom = ['Blood Mix', "Cholesterol"])

In [ ]:
plt.imshow(test_comps[:,:,0], cmap = "hot")
plt.clim([1e5, 5e6])

In [ ]:
plt.figure(figsize = (15, 10))
for i, wave in enumerate(sorted([int(wave[:3]) for wave in os.listdir(f"/content/20230217_Cholesterol_Blood/") if wave[-3:] == 'mat'])):
    plt.subplot(3, 6, i+1)
    plt.imshow(np.mean(np.array(loadmat(f'/content/20230217_Cholesterol_Blood/{wave}nm.mat')['ImgData'])[1][0], axis = 3)[:,:,0], cmap = "hot")
    plt.title(f"{wave} nm")
    plt.colorbar()
plt.show()